# 0. 라이브러리 불러오기

In [738]:
import mido
from mido import MidiFile, MidiTrack
import numpy as np
import pandas as pd
import editdistance
import os

# 1. 동일한 시간에 동시에 누르는 건반에 대해서 데이터 추출하기
- 'Sec' (초): 각 미디 이벤트가 발생한 시간
- 'Ticks' (틱): MIDI 파일에서 사용되는 시간 단위인 틱
- 'msg_type' (이벤트 타입): 각 미디 이벤트의 종류(주로 'note_on', 'note_off', 'set_tempo' 등의 값)
    - 'note_on' 이벤트는 악기에서 음표를 연주할 때 발생하는 이벤트로, 특정 음표가 시작되는 시점을 나타냄
    - 'note_off' 이벤트는 해당 음표의 연주가 끝나는 시점을 나타냄
    - 'set_tempo': 'set_tempo' 이벤트는 MIDI 파일 내에서 템포(속도)를 설정하는 이벤트, 템포는 음악의 속도를 나타내며, 'set_tempo' 이벤트를 통해 BPM(분당 박자) 값이 설정됨
- 'Channel' (채널): MIDI에서 사용되는 채널 번호(미디 이벤트가 어떤 악기 채널에 속하는지를 표시)
- 'Notes' (음표): 노트 이벤트에서 발생한 음표(음 높이) 정보
- 'Velocities' (세기): 노트 이벤트에서 발생한 음의 세기 값
- 'Note_Lengths' (노트 길이): 각 노트 이벤트의 길이(혹은 지속 시간)를 나타내는 열
- 'Tempo' (템포): 템포 이벤트에서 설정된 BPM (분당 박자) 값
- 'mean_velocity' (세기 평균): 각 노트 이벤트에서의 세기 평균을 나타내는 열
    - 셈여림에 활용
- 'diff_velocity' (세기 변화): 각 노트 이벤트에서의 세기의 차이(현재 노트의 평균 세기 - 이전 노트의 평균 세기)
    - 음표의 세기, 빠르기 변화 등에 활용
- 'Dynamic' (셈여림): 셈여림을 나타내는 열, mean_velocity를 활용해서 계산
- 'accent' (악센트): velocity가 76~127, 이전 노트의 velocity 평균보다 20% 더 클 때 accent 1(O), 아니면 0(X)

|셈여림(dynamic)|정의|평균 velocity 범위|
|:--:|:--:|:--:|
|ppp(피아니시시모)|아주 여리게|0~35|
|pp(피아니시모)|매우 여리게|36~48|
|p(피아노)|여리게|49~61|
|mp(메조 피아노)|조금 여리게|62~74|
|mf(메조 포르테)|조금 세게|75~87|
|f(포르테)|세게|88~100|
|ff(포르티시모)|매우 세게|101~113|
|fff(포르티시시모)|아주 세게|114~127|

In [885]:
# 셈여림 계산 함수
def calculate_dynamic(mean_velocity):
    dynamic = ''
    if 0 <= mean_velocity < 36:
        dynamic = 'ppp'
    elif 36 <= mean_velocity < 49:
        dynamic = 'pp'
    elif 49 <= mean_velocity < 62:
        dynamic = 'p'
    elif 62 <= mean_velocity < 75:
        dynamic = 'mp'
    elif 75 <= mean_velocity < 88:
        dynamic = 'mf'
    elif 88 <= mean_velocity < 101:
        dynamic = 'f'
    elif 101 <= mean_velocity < 114:
        dynamic = 'ff'
    elif 114 <= mean_velocity <= 127:
        dynamic = 'fff'
    
    return dynamic

In [886]:
def midi_to_dataframe(file_path):
    mid = mido.MidiFile(file_path)

    columns = ['sec', 'ticks', 'msg_type', 'channel', 'note', 'velocity', 'note_lengths',
               'tempo', 'mean_velocity', 'diff_velocity', 'dynamic', 'accent']  # 'accent' 열 추가
    midi_df = pd.DataFrame(columns=columns)

    current_notes = []
    current_velocities = []
    current_note_lengths = []
    current_tempo = None
    current_ticks = None
    current_time = 0
    current_msg_type = []
    prev_notes = []
    prev_velocities = []

    for msg in mid:
        if msg.type in ['note_on', 'note_off']:
            if msg.time > 0:
                # 이전 노트 및 velocity의 평균 계산
                prev_avg_velocity = sum(prev_velocities) / len(prev_velocities) if prev_velocities else 0

                # 변화 속도 및 평균 속도 계산
                if len(current_velocities) > 0:  # 노트가 있는 경우에만 계산
                    diff_velocity = sum(current_velocities) / len(current_velocities) - prev_avg_velocity
                    mean_velocity = sum(current_velocities) / len(current_velocities)
                else:
                    diff_velocity = 0  # 없으면 0으로 처리
                    mean_velocity = 0

                # 셈여림 계산
                dynamic = calculate_dynamic(mean_velocity)

                # 'accent' 열 추가
                accent = 1 if 76 <= mean_velocity <= 127 and mean_velocity > 1.2 * prev_avg_velocity else 0

                if current_msg_type:
                    # Note_Lengths 리스트에서 [0]번째 값만 사용
                    note_length = current_note_lengths[0] if current_note_lengths else 0
                    midi_df = pd.concat([midi_df, pd.DataFrame([[current_time, current_ticks, current_msg_type, msg.channel, current_notes,
                                                                 current_velocities, note_length, current_tempo, diff_velocity, mean_velocity, dynamic, accent
                                                                 ]], columns=columns)], ignore_index=True)

                prev_notes = current_notes.copy()
                prev_velocities = current_velocities.copy()

                current_notes = []
                current_velocities = []
                current_note_lengths = []
                current_msg_type = []

            current_notes.append(msg.note)
            current_velocities.append(msg.velocity)
            current_note_lengths.append(msg.time)
            current_msg_type.append(msg.type)

        elif msg.type == 'set_tempo':
            current_tempo = mido.tempo2bpm(msg.tempo)  # 템포 정보 업데이트

        current_ticks = msg.time

        current_time += msg.time

    # 마지막 노트들을 처리
    if current_notes and current_msg_type:
        prev_avg_velocity = sum(prev_velocities) / len(prev_velocities) if prev_velocities else 0
        if len(current_velocities) > 0:  # 노트가 있는 경우에만 계산
            diff_velocity = sum(current_velocities) / len(current_velocities) - prev_avg_velocity
            mean_velocity = sum(current_velocities) / len(current_velocities)
        else:
            diff_velocity = 0
            mean_velocity = 0

        # 셈여림 계산
        dynamic = calculate_dynamic(mean_velocity)

        # 'accent' 열 추가
        accent = 1 if 76 <= mean_velocity <= 127 and mean_velocity > 1.2 * prev_avg_velocity else 0

        # Note_Lengths 리스트에서 [0]번째 값만 사용
        note_length = current_note_lengths[0] if current_note_lengths else 0
        midi_df = pd.concat([midi_df, pd.DataFrame([[current_time, current_ticks, current_msg_type, '', current_notes,
                                                     current_velocities, note_length, current_tempo, diff_velocity, mean_velocity, dynamic, accent]], columns=columns)], ignore_index=True)

    return midi_df

In [887]:
def print_midi_info(file_path):
    mid = mido.MidiFile(file_path)

    print("파일이름 :", mid.filename)
    print("총 재생시간 :", mid.length)

    if mid.tracks:
        print('-' * 100)
        print("트랙 이름 :", mid.tracks[0].name)

        # mid.tracks의 길이를 통해 총 트랙의 수 확인
        total_tracks = len(mid.tracks)
        print("총 트랙의 수:", total_tracks)
        for i in range(total_tracks):
            if mid.tracks[i].name:
                print(mid.tracks[i].name)  # 바이트값 출력
            else:
                print("트랙 이름이 없음")

        print('-' * 100)

        for i in range(total_tracks):
            note_count = sum(1 for msg in mid.tracks[i] if msg.type == 'note_on' or msg.type == 'note_off')
            print(f"트랙 {i + 1}의 총 노트 수: {note_count}")
        print('-' * 100)

        for msg in mid:
            print(msg)

    else:
        print("트랙이 없습니다.")

In [888]:
midi_file_path = 'midi_data/Fugue1.mid'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('hello1.csv', index=False)

print(result_df)

파일이름 : midi_data/Fugue1.mid
총 재생시간 : 103.13932137500001
----------------------------------------------------------------------------------------------------
트랙 이름 : 
총 트랙의 수: 5
트랙 이름이 없음
트랙 이름이 없음
트랙 이름이 없음
트랙 이름이 없음
트랙 이름이 없음
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 464
트랙 3의 총 노트 수: 396
트랙 4의 총 노트 수: 350
트랙 5의 총 노트 수: 288
----------------------------------------------------------------------------------------------------
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('set_tempo', tempo=937500, time=0)
MetaMessage('text', text='1', time=0)
MetaMessage('text', text='2', time=0)
MetaMessage('text', text='3', time=0)
MetaMessage('text', text='4', time=0)
note_on channel=1 note=60 velocity=64 time=0.46875
note_off channel=1 note=60 velocity=62 time=0.46875
note_on channel=1 note=62 velocity=64 time=0
note_off channel=1

            sec     ticks                      msg_type channel          note  \
0      0.468750  0.468750                     [note_on]       1          [60]   
1      0.937500  0.000000           [note_off, note_on]       1      [60, 62]   
2      1.406250  0.000000           [note_off, note_on]       1      [62, 64]   
3      1.875000  0.000000           [note_off, note_on]       1      [64, 65]   
4      2.578125  0.000000           [note_off, note_on]       1      [65, 67]   
..          ...       ...                           ...     ...           ...   
765   99.981427  0.000000  [note_off, note_on, note_on]       3  [74, 79, 76]   
766  102.961690  2.980262                    [note_off]       1          [48]   
767  103.001164  0.039474                    [note_off]       0          [60]   
768  103.119585  0.118421                    [note_off]       1          [84]   
769  103.139321  0.000000          [note_off, note_off]              [76, 79]   

         velocity  note_len

In [889]:
midi_file_path = 'midi_data/Fugue3.mid'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('hello2.csv', index=False)

print(result_df)

파일이름 : midi_data/Fugue3.mid
총 재생시간 : 162.20662299999861
----------------------------------------------------------------------------------------------------
트랙 이름 : 
총 트랙의 수: 4
트랙 이름이 없음
트랙 이름이 없음
트랙 이름이 없음
트랙 이름이 없음
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 1218
트랙 3의 총 노트 수: 670
트랙 4의 총 노트 수: 1004
----------------------------------------------------------------------------------------------------
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('set_tempo', tempo=731707, time=0)
MetaMessage('text', text='RH H', time=0)
MetaMessage('text', text='RH L', time=0)
MetaMessage('text', text='LH L', time=0)
note_on channel=0 note=68 velocity=64 time=1.0975605
note_off channel=0 note=68 velocity=80 time=0.3658535
note_on channel=0 note=70 velocity=64 time=0
note_off channel=0 note=70 velocity=74 time=0.18292675
note_on channe

            sec     ticks                                           msg_type  \
0      1.097560  1.097560                                          [note_on]   
1      1.463414  0.000000                                [note_off, note_on]   
2      1.646341  0.000000                                [note_off, note_on]   
3      1.829268  0.000000                                [note_off, note_on]   
4      2.012194  0.000000                                [note_off, note_on]   
..          ...       ...                                                ...   
892  159.584519  0.000000                     [note_off, note_off, note_off]   
893  159.723864  0.098361               [note_on, note_on, note_on, note_on]   
894  160.094554  0.000000  [note_off, note_off, note_off, note_off, note_...   
895  160.137657  0.000000      [note_on, note_on, note_on, note_on, note_on]   
896  162.206623  0.000000  [note_off, note_off, note_off, note_off, note_...   

    channel                  note      

In [890]:
midi_file_path = 'midi_data/Prelude1.mid'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('hello3.csv', index=False)

print(result_df)

파일이름 : midi_data/Prelude1.mid
총 재생시간 : 117.95786391249983
----------------------------------------------------------------------------------------------------
트랙 이름 : 
총 트랙의 수: 3
트랙 이름이 없음
트랙 이름이 없음
트랙 이름이 없음
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 834
트랙 3의 총 노트 수: 268
----------------------------------------------------------------------------------------------------
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('set_tempo', tempo=833333, time=0)
MetaMessage('text', text='RH B', time=0)
MetaMessage('text', text='LH B', time=0)
note_on channel=0 note=60 velocity=64 time=0
note_on channel=0 note=64 velocity=64 time=0.20833325
note_on channel=0 note=67 velocity=64 time=0.20833325
note_off channel=0 note=67 velocity=44 time=0.20833325
note_on channel=0 note=72 velocity=64 time=0
note_off channel=0 note=72 velocity=7

            sec     ticks                        msg_type channel  \
0             0         0                       [note_on]       0   
1      0.208333  0.208333                       [note_on]       0   
2      0.416666  0.208333                       [note_on]       0   
3         0.625         0             [note_off, note_on]       0   
4      0.833333         0             [note_off, note_on]       0   
..          ...       ...                             ...     ...   
725  114.419671  0.013889                      [note_off]       0   
726  114.579393  0.159722                      [note_off]       0   
727  114.624532         0              [note_on, note_on]       0   
728  117.322447         0  [note_off, note_off, note_off]       0   
729  117.957864         0            [note_off, note_off]           

             note      velocity note_lengths      tempo  mean_velocity  \
0            [60]          [64]            0  72.000029      64.000000   
1            [64]      

In [891]:
midi_file_path = 'midi_data/sample_midi.mid'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('hello4.csv', index=False)

print(result_df)

파일이름 : midi_data/sample_midi.mid
총 재생시간 : 59.000000000000014
----------------------------------------------------------------------------------------------------
트랙 이름 : Chill The World Chords
총 트랙의 수: 1
Chill The World Chords
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 152
----------------------------------------------------------------------------------------------------
MetaMessage('channel_prefix', channel=0, time=2.0)
MetaMessage('track_name', name='Chill The World Chords', time=0)
MetaMessage('instrument_name', name='Chill The World Chords', time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('key_signature', key='C', time=0)
MetaMessage('smpte_offset', frame_rate=25, hours=1, minutes=0, seconds=0, frames=0, sub_frames=0, time=0)
MetaMessage('set_tempo', tempo=750000, time=0)
note_on channel=0 note=41 velocity=70 time=0
note_

In [892]:
midi_file_path = 'midi_data/hangyul_input.midi'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('input.csv', index=False)

print(result_df)

파일이름 : midi_data/hangyul_input.midi
총 재생시간 : 174.7791666666652
----------------------------------------------------------------------------------------------------
트랙 이름 : test_rivers_flow
총 트랙의 수: 4
test_rivers_flow
MIDI
Piano
Yamaha MOX
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 0
트랙 3의 총 노트 수: 2564
트랙 4의 총 노트 수: 0
----------------------------------------------------------------------------------------------------
MetaMessage('track_name', name='test_rivers_flow', time=0)
MetaMessage('set_tempo', tempo=1000000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('track_name', name='MIDI', time=0)
MetaMessage('key_signature', key='A', time=0)
MetaMessage('track_name', name='Piano', time=0)
MetaMessage('track_name', name='Yamaha MOX', time=0)
note_on channel=0 note=80 velocity=36 time=0.5270833333333333
note_on cha

note_on channel=0 note=52 velocity=0 time=0
note_on channel=0 note=64 velocity=44 time=0.46458333333333335
note_on channel=0 note=64 velocity=44 time=0.0020833333333333333
note_on channel=0 note=81 velocity=59 time=0.016666666666666666
note_on channel=0 note=81 velocity=59 time=0
note_on channel=0 note=59 velocity=0 time=0.0375
note_on channel=0 note=59 velocity=0 time=0.0020833333333333333
note_on channel=0 note=64 velocity=0 time=0.2
note_on channel=0 note=64 velocity=0 time=0
note_on channel=0 note=83 velocity=59 time=0.052083333333333336
note_on channel=0 note=83 velocity=59 time=0.0020833333333333333
note_on channel=0 note=81 velocity=0 time=0.016666666666666666
note_on channel=0 note=81 velocity=0 time=0
note_on channel=0 note=81 velocity=55 time=0.24583333333333332
note_on channel=0 note=81 velocity=55 time=0
note_on channel=0 note=83 velocity=0 time=0.008333333333333333
note_on channel=0 note=83 velocity=0 time=0
note_on channel=0 note=80 velocity=56 time=0.22916666666666666
no

             sec     ticks            msg_type channel      note  velocity  \
0       0.527083  0.527083           [note_on]       0      [80]      [36]   
1       0.529167  0.002083           [note_on]       0      [80]      [36]   
2       0.537500  0.008333           [note_on]       0      [61]      [17]   
3       0.539583  0.002083           [note_on]       0      [61]      [17]   
4       0.843750  0.000000  [note_on, note_on]       0  [61, 61]    [0, 0]   
...          ...       ...                 ...     ...       ...       ...   
1640  174.058333  0.000000  [note_on, note_on]       0  [57, 57]    [0, 0]   
1641  174.075000  0.000000  [note_on, note_on]       0  [81, 81]    [0, 0]   
1642  174.218750  0.000000  [note_on, note_on]       0  [62, 62]    [0, 0]   
1643  174.277083  0.000000  [note_on, note_on]       0  [69, 69]  [30, 30]   
1644  174.779167  0.000000  [note_on, note_on]          [69, 69]    [0, 0]   

      note_lengths  tempo  mean_velocity  diff_velocity dynamic

In [893]:
midi_file_path = 'midi_data/hangyul_target.mid'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('target.csv', index=False)

print(result_df)

파일이름 : midi_data/hangyul_target.mid
총 재생시간 : 164.77394462499913
----------------------------------------------------------------------------------------------------
트랙 이름 : 
총 트랙의 수: 2
트랙 이름이 없음
Piano
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 1600
----------------------------------------------------------------------------------------------------
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('key_signature', key='A', time=0)
MetaMessage('set_tempo', tempo=923075, time=0)
control_change channel=0 control=0 value=0 time=0
control_change channel=0 control=32 value=0 time=0
program_change channel=0 program=0 time=0
MetaMessage('track_name', name='Piano', time=0)
note_on channel=0 note=81 velocity=70 time=0
note_on channel=0 note=54 velocity=72 time=0
note_on channel=0 note=81 velocity=0 time=0.4615375
note_on channel=0 

            sec     ticks                                           msg_type  \
0             0         0                                 [note_on, note_on]   
1      0.461537         0               [note_on, note_on, note_on, note_on]   
2      0.923075         0               [note_on, note_on, note_on, note_on]   
3      1.384613         0                                 [note_on, note_on]   
4       1.84615         0               [note_on, note_on, note_on, note_on]   
..          ...       ...                                                ...   
516  158.076259         0                                 [note_on, note_on]   
517  158.537796         0               [note_on, note_on, note_on, note_on]   
518  159.460869         0               [note_on, note_on, note_on, note_on]   
519  160.383942         0  [note_on, note_on, note_on, note_on, note_on, ...   
520  164.773945  0.005409  [note_on, note_on, note_on, note_on, note_on, ...   

    channel                            

In [894]:
midi_file_path = 'midi_data/hangyul_input2_1.midi'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('input2_1.csv', index=False)

print(result_df)

파일이름 : midi_data/hangyul_input2_1.midi
총 재생시간 : 12.470833333333331
----------------------------------------------------------------------------------------------------
트랙 이름 : test_rivers_flow
총 트랙의 수: 3
test_rivers_flow
Piano
Yamaha MOX
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 108
트랙 3의 총 노트 수: 0
----------------------------------------------------------------------------------------------------
MetaMessage('track_name', name='test_rivers_flow', time=0)
MetaMessage('set_tempo', tempo=1000000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('track_name', name='Piano', time=0)
MetaMessage('track_name', name='Yamaha MOX', time=0)
note_on channel=0 note=72 velocity=53 time=2.8125
note_on channel=0 note=72 velocity=53 time=0
note_on channel=0 note=76 velocity=75 time=0.32708333333333334
note_on channel=0 note=76 

In [895]:
midi_file_path = 'midi_data/hangyul_input2_2.midi'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('input2_2.csv', index=False)

print(result_df)

파일이름 : midi_data/hangyul_input2_2.midi
총 재생시간 : 12.458333333333329
----------------------------------------------------------------------------------------------------
트랙 이름 : test_rivers_flow
총 트랙의 수: 3
test_rivers_flow
Piano
Yamaha MOX
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 96
트랙 3의 총 노트 수: 0
----------------------------------------------------------------------------------------------------
MetaMessage('track_name', name='test_rivers_flow', time=0)
MetaMessage('set_tempo', tempo=1000000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('track_name', name='Piano', time=0)
MetaMessage('track_name', name='Yamaha MOX', time=0)
note_on channel=0 note=72 velocity=59 time=2.7916666666666665
note_on channel=0 note=72 velocity=59 time=0
note_on channel=0 note=76 velocity=68 time=0.32708333333333334
note_on channel

In [896]:
midi_file_path = 'midi_data/hangyul_input2_3.midi'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('input2_3.csv', index=False)

print(result_df)

파일이름 : midi_data/hangyul_input2_3.midi
총 재생시간 : 12.583333333333334
----------------------------------------------------------------------------------------------------
트랙 이름 : test_rivers_flow
총 트랙의 수: 3
test_rivers_flow
Piano
Yamaha MOX
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 92
트랙 3의 총 노트 수: 0
----------------------------------------------------------------------------------------------------
MetaMessage('track_name', name='test_rivers_flow', time=0)
MetaMessage('set_tempo', tempo=1000000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('track_name', name='Piano', time=0)
MetaMessage('track_name', name='Yamaha MOX', time=0)
note_on channel=0 note=71 velocity=66 time=2.85625
note_on channel=0 note=71 velocity=66 time=0.0020833333333333333
note_on channel=0 note=76 velocity=53 time=0.30416666666666664
note_o

In [897]:
midi_file_path = 'midi_data/hangyul_target2.midi'

# 미디 파일의 트랙 정보 출력
print_midi_info(midi_file_path)

result_df = midi_to_dataframe(midi_file_path)
result_df.to_csv('target2.csv', index=False)

print(result_df)

파일이름 : midi_data/hangyul_target2.midi
총 재생시간 : 12.424999999999997
----------------------------------------------------------------------------------------------------
트랙 이름 : test_rivers_flow
총 트랙의 수: 3
test_rivers_flow
Yamaha MOX
Yamaha MOX
----------------------------------------------------------------------------------------------------
트랙 1의 총 노트 수: 0
트랙 2의 총 노트 수: 108
트랙 3의 총 노트 수: 0
----------------------------------------------------------------------------------------------------
MetaMessage('track_name', name='test_rivers_flow', time=0)
MetaMessage('set_tempo', tempo=1000000, time=0)
MetaMessage('time_signature', numerator=4, denominator=4, clocks_per_click=24, notated_32nd_notes_per_beat=8, time=0)
MetaMessage('track_name', name='Yamaha MOX', time=0)
MetaMessage('track_name', name='Yamaha MOX', time=0)
note_on channel=0 note=72 velocity=79 time=2.7604166666666665
note_on channel=1 note=72 velocity=79 time=0.0020833333333333333
note_on channel=0 note=76 velocity=59 time=0.333

# 2. 알고리즘 설계

## 2_1. 사전 작업
- 입력 데이터 파일(input_file)과 정답 데이터 파일(target_file)을 비교해서 무조건 정답 데이터 파일의 행 길이로 맞추기
- 만약에 입력 데이터 파일의 행 길이가 작으면 나머지는 0으로 처리
- Precheck

## 2_2. 노트 정확도(음정 정확도: pitch accuracy)
- 입력 데이터와 정답 데이터의 Notes를 비교하여 현재 노트가 일치하면 2점을 부여
- 구체화 필요
    - ex: 실수로 잘못 쳐서 한번 밀려서 연주한 경우 계속 밀리는 문제(-1, +1번째에 대해서도 검사를 할 것인지?) 
    
## 2_3_1. 셈여림
- 로직
<pre>
dynamic = ''
if 0 <= mean_velocity <= 35:
    dynamic = 'ppp'
elif 36 <= mean_velocity <= 48:
    dynamic = 'pp'
elif 49 <= mean_velocity <= 61:
    dynamic = 'p'
elif 62 <= mean_velocity <= 74:
    dynamic = 'mp'
elif 75 <= mean_velocity <= 87:
    dynamic = 'mf'
elif 88 <= mean_velocity <= 100:
    dynamic = 'f'
elif 101 <= mean_velocity <= 113:
    dynamic = 'ff'
elif 114 <= mean_velocity <= 127:
    dynamic = 'fff'
</pre>
## 2_3_2. 셈여림의 변화

## 2_4_1. 곡의 빠르기 유사도 점수
## 2_4_2. 빠르기의 변화
## 2_5. 붙임줄, 스타카토, 테누토, 늘임표
## 2_6. 악센트
## 2_7. 옥타브
## 2_8. 꾸밈음, 반복 기호
## 2_9. 리듬
## 2_10. 페달링
- SUSTAIN PEDAL: if msg.control == 64:
## 2_11. 노트의 길이 유사도 점수 구하기

# 3. 알고리즘 구현

In [1281]:
# 입력 데이터 및 정답 데이터 불러오기
input_file = 'hello1.csv'
input_file2 = 'hello3.csv'
input_file3 = 'hello4.csv'
target_file = 'hello2.csv'

input_hangyul = 'input.csv'
target_hangyul = 'target.csv'
input_hangyul1 = 'input2_1.csv'
input_hangyul2 = 'input2_2.csv'
input_hangyul3 = 'input2_3.csv'
target_hangyul = 'target.csv'
target_hangyul2 = 'target2.csv'

new_input = 'new_input.csv'
new_target = 'new_target.csv'

a_input = 'a_input.csv'
a_target = 'a_target.csv'

In [1282]:
def pre_check(input_mid, target_mid):
    input_track_len  = len(input_mid.tracks)
    target_track_len = len(target_mid.tracks)
    
    # track 길이 비교
    if input_track_len != target_track_len:
        print(f"input track len({input_track_len}) != target track len({target_track_len})")
        return False
    
    # track_name 비교
    input_track_name  = [ msg.name for msg in input_mid if msg.type == 'track_name']
    target_track_name = [ msg.name for msg in target_mid if msg.type == 'track_name']
    
    ## if input_track_name != target_track_name:
    if input_track_name == target_track_name:
        print(f"input track name({input_track_name}) != target track name({target_track_name})")
        return False
    
    # track 시간 비교 ( time_diff_interval 이상 비교 불가)
    input_mid_time  = input_mid.length
    target_mid_time = target_mid.length
    
    time_diff_interval = 20
    
    if abs(input_mid_time - target_mid_time) > time_diff_interval:
        print(f"input time({input_mid_time}) != target time({target_mid_time})")
        return False
    
    return True

## 3_1. 두 파일의 길이 맞추기

In [1283]:
def align_and_save_dataframes(input_file, target_file):
    df_input = pd.read_csv(input_file)
    df_target = pd.read_csv(target_file)

    # target_file의 행 길이로 맞추기
    min_length = min(len(df_input), len(df_target))
    df_input_aligned = df_input.head(min_length)

    # target_file보다 행이 작으면 나머지 행은 0으로 채우기
    if len(df_input) < len(df_target):
        df_input_aligned = pd.concat([df_input_aligned, pd.DataFrame(0, index=range(len(df_target) - len(df_input)), columns=df_input.columns)])

    # 모든 컬럼에 대해 0으로 채우기
    for col in df_input.columns:
        if col not in df_target.columns:
            df_input_aligned[col] = 0

    # msg_type, note, velocity 컬럼에 대해 빈 리스트 []를 NaN으로 처리
    for col in ['msg_type', 'note', 'velocity']:
        df_input_aligned[col] = df_input_aligned[col].apply(lambda x: np.nan if x == '[]' else x)

    df_input_aligned.to_csv(input_file, index=False)
    df_target.to_csv(target_file, index=False)

In [1284]:
align_and_save_dataframes(input_file, target_file)
align_and_save_dataframes(input_file2, target_file)
align_and_save_dataframes(input_file3, target_file)
align_and_save_dataframes(input_hangyul, target_hangyul)
align_and_save_dataframes(input_hangyul1, target_hangyul2)
align_and_save_dataframes(input_hangyul2, target_hangyul2)
align_and_save_dataframes(input_hangyul3, target_hangyul2)

In [1285]:
align_and_save_dataframes(new_input, new_target)
align_and_save_dataframes(a_input, a_target)

C:\Users\User\AppData\Local\Temp\ipykernel_10484\2677449900.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input_aligned[col] = df_input_aligned[col].apply(lambda x: np.nan if x == '[]' else x)
C:\Users\User\AppData\Local\Temp\ipykernel_10484\2677449900.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_input_aligned[col] = df_input_aligned[col].apply(lambda x: np.nan if x == '[]' else x)
C:\Users\User\AppData\Local\Temp\ipykernel_10484\2677449900.py:20: SettingWithCopyWarning: 
A value is try

## 3_2. 노트 정확도(음정 정확도:note(pitch) accuracy)

In [1286]:
def calculate_note_accuracy(input_file, target_file):
    df_input = pd.read_csv(input_file)
    df_target = pd.read_csv(target_file)

    total_accuracy = 0

    for index in range(len(df_input)):
        notes_input = eval(df_input.at[index, 'note'])
        notes_target = eval(df_target.at[index, 'note'])

        # 0으로 된 값 리스트로 처리
        if isinstance(notes_input, int):
            notes_input = [notes_input]
        if isinstance(notes_target, int):
            notes_target = [notes_target]
        
        row_accuracy = 0

        for note_input in set(notes_input):
            for note_target in set(notes_target):
                if note_input == note_target:
                    row_accuracy += 1
                    print('-'*20)
                    print(f'입력 데이터: {notes_input}')
                    print(f'정답 데이터: {notes_target}')
                    print(f'!!!{row_accuracy}점 추가!!!')
                    print('-'*20)

                print(f'비교 데이터 - 입력 데이터: {note_input} / 정답 데이터: {note_target}')
                pass

        total_accuracy += row_accuracy
        print(f'현재 점수: {total_accuracy}점')
        print()

    max_possible_accuracy = len(df_input) * max(len(set(notes_input)), len(set(notes_target)))
    accuracy_percentage = min(100, (total_accuracy / max_possible_accuracy) * 100)

    return f'{accuracy_percentage:.2f}%'

accuracy = calculate_note_accuracy(input_hangyul3, target_hangyul2)

print(f'음정 정확도: {accuracy}')

비교 데이터 - 입력 데이터: 71 / 정답 데이터: 72
현재 점수: 0점

비교 데이터 - 입력 데이터: 71 / 정답 데이터: 72
현재 점수: 0점

--------------------
입력 데이터: [76, 76]
정답 데이터: [76]
!!!1점 추가!!!
--------------------
비교 데이터 - 입력 데이터: 76 / 정답 데이터: 76
현재 점수: 1점

비교 데이터 - 입력 데이터: 71 / 정답 데이터: 76
현재 점수: 1점

비교 데이터 - 입력 데이터: 79 / 정답 데이터: 72
현재 점수: 1점

--------------------
입력 데이터: [79]
정답 데이터: [79, 79]
!!!1점 추가!!!
--------------------
비교 데이터 - 입력 데이터: 79 / 정답 데이터: 79
현재 점수: 2점

--------------------
입력 데이터: [76]
정답 데이터: [76, 76]
!!!1점 추가!!!
--------------------
비교 데이터 - 입력 데이터: 76 / 정답 데이터: 76
현재 점수: 3점

비교 데이터 - 입력 데이터: 76 / 정답 데이터: 71
현재 점수: 3점

--------------------
입력 데이터: [79, 79]
정답 데이터: [79, 79]
!!!1점 추가!!!
--------------------
비교 데이터 - 입력 데이터: 79 / 정답 데이터: 79
현재 점수: 4점

비교 데이터 - 입력 데이터: 69 / 정답 데이터: 76
현재 점수: 4점

비교 데이터 - 입력 데이터: 69 / 정답 데이터: 76
현재 점수: 4점

비교 데이터 - 입력 데이터: 69 / 정답 데이터: 71
현재 점수: 4점

비교 데이터 - 입력 데이터: 76 / 정답 데이터: 71
현재 점수: 4점

--------------------
입력 데이터: [79, 79]
정답 데이터: [79, 79]
!!!1점 추가!!!
--------------------


In [1287]:
def calculate_note_accuracy(input_file, target_file):
    df_input = pd.read_csv(input_file)
    df_target = pd.read_csv(target_file)

    total_accuracy = 0

    for index in range(len(df_input)):
        notes_input = eval(df_input.at[index, 'note'])
        notes_target = eval(df_target.at[index, 'note'])

        # 0으로 된 값 리스트로 처리
        if isinstance(notes_input, int):
            notes_input = [notes_input]
        if isinstance(notes_target, int):
            notes_target = [notes_target]
        
        row_accuracy = 0

        for note_input in set(notes_input):
            for note_target in set(notes_target):
                if note_input == 0: # 0이면 계산에 포함하지 않음
                    pass
                
                # 음정 차이 계산 - 노트 번호 차이에 따라 가중치 부여
                pitch_difference = abs(note_input - note_target)
                
                # 약간의 차이에도 점수 부여
                if pitch_difference <= 1:
                    row_accuracy += 2
                elif pitch_difference <= 2:
                    row_accuracy += 1  # 1차적인 음정 차이보다 크지만 약간의 차이라면 0.5점 부여
                elif pitch_difference <= 3:
                    row_accuracy += 0.5  # 1차적인 음정 차이보다 크지만 약간의 차이라면 0.5점 부여
                pass

        total_accuracy += row_accuracy

    max_possible_accuracy = len(df_input) * max(len(set(notes_input)), len(set(notes_target)))
    accuracy_percentage = min(100, (total_accuracy / max_possible_accuracy) * 100)
    # 나중에 각 노트의 개수를 센 후, 개수의 차이에 따라 가중치를 더해주는 것도 괜찮을 것 같음

    return f'{accuracy_percentage:.2f}%'

accuracy = calculate_note_accuracy(input_hangyul3, target_hangyul2)

print(f'{target_hangyul2}에 대한 {input_hangyul3}의 음정 정확도: {accuracy}')

target2.csv에 대한 input2_3.csv의 음정 정확도: 41.78%


In [1293]:
# 최종 코드

def calculate_note_accuracy(input_file, target_file):
    df_input = pd.read_csv(input_file)
    df_target = pd.read_csv(target_file)

    total_accuracy = 0

    for index in range(len(df_input)):
        notes_input = eval(df_input.at[index, 'note'])
        notes_target = eval(df_target.at[index, 'note'])

        # 0으로 된 값 리스트로 처리
        if isinstance(notes_input, int):
            notes_input = [notes_input]
        if isinstance(notes_target, int):
            notes_target = [notes_target]
        
        row_accuracy = 0

        for note_input in set(notes_input):
            for note_target in set(notes_target):
                if note_input == 0: # 0이면 계산에 포함하지 않음
                    pass
                
                # 음정 차이 계산 - 노트 번호 차이에 따라 가중치 부여
                pitch_difference = abs(note_input - note_target)
                
                # 약간의 차이에도 점수 부여
                if pitch_difference <= 1:
                    row_accuracy += 2
                elif pitch_difference <= 2:
                    row_accuracy += 1  # 1차적인 음정 차이보다 크지만 약간의 차이라면 0.5점 부여
                elif pitch_difference <= 3:
                    row_accuracy += 0.5  # 1차적인 음정 차이보다 크지만 약간의 차이라면 0.5점 부여
                pass

        total_accuracy += row_accuracy

    max_possible_accuracy = len(df_input) * max(len(set(notes_input)), len(set(notes_target)))
    accuracy_percentage = min(100, (total_accuracy / max_possible_accuracy) * 100)
    # 나중에 각 노트의 개수를 센 후, 개수의 차이에 따라 가중치를 더해주는 것도 괜찮을 것 같음

    return f'{accuracy_percentage:.2f}%'

In [1294]:
# 음정 정확도 Score 계산
note_score1 = calculate_note_accuracy(input_hangyul1, target_hangyul2)
note_score2 = calculate_note_accuracy(input_hangyul2, target_hangyul2)
note_score3 = calculate_note_accuracy(input_hangyul3, target_hangyul2)
note_score4 = calculate_note_accuracy(target_hangyul2, target_hangyul2)
note_score5 = calculate_note_accuracy(input_file, target_file)
note_score6 = calculate_note_accuracy(input_file2, target_file)
note_score7 = calculate_note_accuracy(input_file3, target_file)
note_score8 = calculate_note_accuracy(a_input, a_target)

print(f"input: {input_hangyul1} | target: {target_hangyul2} | 곡의 음정 정확도 Score1 | {note_score1}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 곡의 음정 정확도 Score2 | {note_score2}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 곡의 음정 정확도 Score3 | {note_score3}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 곡의 음정 정확도 Score4 | {note_score4}")
print(f"input: {input_file} | target: {target_file} | 곡의 음정 정확도 Score5 | {note_score5}")
print(f"input: {input_file2} | target: {target_file} | 곡의 음정 정확도 Score6 | {note_score6}")
print(f"input: {input_file3} | target: {target_file} | 곡의 음정 정확도 Score7 | {note_score7}")
print(f"input: {a_input} | target: {a_target} | 곡의 음정 정확도 Score8 | {note_score8}")

input: input2_1.csv | target: target2.csv | 곡의 음정 정확도 Score1 | 84.25%
input: input2_2.csv | target: target2.csv | 곡의 음정 정확도 Score2 | 36.30%
input: input2_3.csv | target: target2.csv | 곡의 음정 정확도 Score3 | 41.78%
input: target2.csv | target: target2.csv | 곡의 음정 정확도 Score4 | 100.00%
input: hello1.csv | target: hello2.csv | 곡의 음정 정확도 Score5 | 25.50%
input: hello3.csv | target: hello2.csv | 곡의 음정 정확도 Score6 | 20.42%
input: hello4.csv | target: hello2.csv | 곡의 음정 정확도 Score7 | 1.45%
input: a_input.csv | target: a_target.csv | 곡의 음정 정확도 Score8 | 40.13%


## 3_3_1. 셈여림 유사도

In [1289]:
def calculate_similarity_dynamic(input_file, target_file):
    input_df = pd.read_csv(input_file)
    target_df = pd.read_csv(target_file)

    input_counts = input_df['dynamic'].value_counts().to_dict()
    target_counts = target_df['dynamic'].value_counts().to_dict()

    return input_counts, target_counts

# 셈여림
dynamic_note_score1 = calculate_similarity_dynamic(input_hangyul1, target_hangyul2)
dynamic_note_score2 = calculate_similarity_dynamic(input_hangyul2, target_hangyul2)
dynamic_note_score3 = calculate_similarity_dynamic(input_hangyul3, target_hangyul2)
dynamic_note_score4 = calculate_similarity_dynamic(target_hangyul2, target_hangyul2)
dynamic_note_score5 = calculate_similarity_dynamic(input_file, target_file)
dynamic_note_score6 = calculate_similarity_dynamic(input_file2, target_file)
dynamic_note_score7 = calculate_similarity_dynamic(input_file3, target_file)
dynamic_note_score8 = calculate_similarity_dynamic(a_input, a_target)

print(f"input: {input_hangyul1} | target: {target_hangyul2} | 곡의 셈여림 카운트 | \n{dynamic_note_score1}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 곡의 셈여림 카운트 | \n{dynamic_note_score2}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 곡의 셈여림 카운트 | \n{dynamic_note_score3}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 곡의 셈여림 카운트 | \n{dynamic_note_score4}")
print(f"input: {input_file} | target: {target_file} | 곡의 셈여림 카운트 | \n{dynamic_note_score5}")
print(f"input: {input_file2} | target: {target_file} | 곡의 셈여림 카운트 | \n{dynamic_note_score6}")
print(f"input: {input_file3} | target: {target_file} | 곡의 셈여림 카운트 | \n{dynamic_note_score7}")
print(f"input: {a_input} | target: {a_target} | 곡의 셈여림 카운트 | \n{dynamic_note_score8}")

input: input2_1.csv | target: target2.csv | 곡의 셈여림 카운트 | 
({'ppp': 31, 'mp': 17, 'mf': 16, 'p': 5, '0': 4}, {'ppp': 33, 'mf': 22, 'mp': 13, 'p': 5})
input: input2_2.csv | target: target2.csv | 곡의 셈여림 카운트 | 
({'ppp': 26, 'mf': 17, 'mp': 15, '0': 12, 'f': 2, 'p': 1}, {'ppp': 33, 'mf': 22, 'mp': 13, 'p': 5})
input: input2_3.csv | target: target2.csv | 곡의 셈여림 카운트 | 
({'ppp': 25, '0': 17, 'p': 13, 'mp': 10, 'mf': 8}, {'ppp': 33, 'mf': 22, 'mp': 13, 'p': 5})
input: target2.csv | target: target2.csv | 곡의 셈여림 카운트 | 
({'ppp': 33, 'mf': 22, 'mp': 13, 'p': 5}, {'ppp': 33, 'mf': 22, 'mp': 13, 'p': 5})
input: hello1.csv | target: hello2.csv | 곡의 셈여림 카운트 | 
({'mp': 416, 'p': 322, '0': 127, 'pp': 17, 'mf': 11, 'ppp': 4}, {'mp': 558, 'p': 304, 'pp': 20, 'mf': 8, 'ppp': 7})
input: hello3.csv | target: hello2.csv | 곡의 셈여림 카운트 | 
({'mp': 355, 'p': 307, '0': 167, 'pp': 58, 'mf': 8, 'ppp': 2}, {'mp': 558, 'p': 304, 'pp': 20, 'mf': 8, 'ppp': 7})
input: hello4.csv | target: hello2.csv | 곡의 셈여림 카운트 | 
({'0': 

In [1323]:
def calculate_similarity_dynamic(input_file, target_file):
    input_df = pd.read_csv(input_file)
    target_df = pd.read_csv(target_file)

    input_counts = input_df['dynamic'].value_counts().to_dict()
    target_counts = target_df['dynamic'].value_counts().to_dict()
    
    print('-'*50)
    print(f'{input_file} vs {target_file}')
    print(input_counts)
    print(target_counts)
    print('-'*50)

    # 공통된 'Dynamic' 값들의 유사도 계산
    common_keys = set(input_counts) & set(target_counts)
    
    for key in common_keys:
        print(f'input: {key} - {input_counts.get(key, 0)}개')
        print(f'target: {key} - {target_counts.get(key, 0)}개')
        print('-'*50)
    print('-'*100)

    # 총 가능한 차이의 합 및 실제 차이의 합 계산
    total_possible_difference = sum(max(input_counts.get(key, 0), target_counts.get(key, 0)) for key in common_keys)
    actual_difference = sum(abs(input_counts.get(key, 0) - target_counts.get(key, 0)) for key in common_keys)

    # ((총 가능한 차이의 합 - 실제 차이의 합) / 총 가능한 차이의 합) * 100
    similarity_percentage = ((total_possible_difference - actual_difference) / total_possible_difference) * 100
    return f'{similarity_percentage:.2f}%'

In [1324]:
# 셈여림
dynamic_note_score1 = calculate_similarity_dynamic(input_hangyul1, target_hangyul2)
dynamic_note_score2 = calculate_similarity_dynamic(input_hangyul2, target_hangyul2)
dynamic_note_score3 = calculate_similarity_dynamic(input_hangyul3, target_hangyul2)
dynamic_note_score4 = calculate_similarity_dynamic(target_hangyul2, target_hangyul2)
dynamic_note_score5 = calculate_similarity_dynamic(input_file, target_file)
dynamic_note_score6 = calculate_similarity_dynamic(input_file2, target_file)
dynamic_note_score7 = calculate_similarity_dynamic(input_file3, target_file)
dynamic_note_score8 = calculate_similarity_dynamic(a_input, a_target)

print(f"input: {input_hangyul1} | target: {target_hangyul2} | 곡의 셈여림 유사도 | {dynamic_note_score1}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 곡의 셈여림 유사도 | {dynamic_note_score2}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 곡의 셈여림 유사도 | {dynamic_note_score3}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 곡의 셈여림 유사도 | {dynamic_note_score4}")
print(f"input: {input_file} | target: {target_file} | 곡의 셈여림 유사도 | {dynamic_note_score5}")
print(f"input: {input_file2} | target: {target_file} | 곡의 셈여림 유사도 | {dynamic_note_score6}")
print(f"input: {input_file3} | target: {target_file} | 곡의 셈여림 유사도 | {dynamic_note_score7}")
print(f"input: {a_input} | target: {a_target} | 곡의 셈여림 카운트 | {dynamic_note_score8}")

--------------------------------------------------
input2_1.csv vs target2.csv
{'ppp': 31, 'mp': 17, 'mf': 16, 'p': 5, '0': 4}
{'ppp': 33, 'mf': 22, 'mp': 13, 'p': 5}
--------------------------------------------------
input: mf - 16개
target: mf - 22개
--------------------------------------------------
input: ppp - 31개
target: ppp - 33개
--------------------------------------------------
input: mp - 17개
target: mp - 13개
--------------------------------------------------
input: p - 5개
target: p - 5개
--------------------------------------------------
----------------------------------------------------------------------------------------------------
--------------------------------------------------
input2_2.csv vs target2.csv
{'ppp': 26, 'mf': 17, 'mp': 15, '0': 12, 'f': 2, 'p': 1}
{'ppp': 33, 'mf': 22, 'mp': 13, 'p': 5}
--------------------------------------------------
input: mf - 17개
target: mf - 22개
--------------------------------------------------
input: ppp - 26개
target: ppp - 33개
-

In [1325]:
# 최종 코드

def calculate_similarity_dynamic(input_file, target_file):
    input_df = pd.read_csv(input_file)
    target_df = pd.read_csv(target_file)

    input_counts = input_df['dynamic'].value_counts().to_dict()
    target_counts = target_df['dynamic'].value_counts().to_dict()

    # 공통된 'Dynamic' 값들의 유사도 계산
    common_keys = set(input_counts) & set(target_counts)

    # 총 가능한 차이의 합 및 실제 차이의 합 계산
    total_possible_difference = sum(max(input_counts.get(key, 0), target_counts.get(key, 0)) for key in common_keys)
    actual_difference = sum(abs(input_counts.get(key, 0) - target_counts.get(key, 0)) for key in common_keys)

    # ((총 가능한 차이의 합 - 실제 차이의 합) / 총 가능한 차이의 합) * 100
    similarity_percentage = ((total_possible_difference - actual_difference) / total_possible_difference) * 100
    return f'{similarity_percentage:.2f}%'

In [1332]:
# 셈여림
dynamic_note_score1 = calculate_similarity_dynamic(input_hangyul1, target_hangyul2)
dynamic_note_score2 = calculate_similarity_dynamic(input_hangyul2, target_hangyul2)
dynamic_note_score3 = calculate_similarity_dynamic(input_hangyul3, target_hangyul2)
dynamic_note_score4 = calculate_similarity_dynamic(target_hangyul2, target_hangyul2)
dynamic_note_score5 = calculate_similarity_dynamic(input_file, target_file)
dynamic_note_score6 = calculate_similarity_dynamic(input_file2, target_file)
dynamic_note_score7 = calculate_similarity_dynamic(input_file3, target_file)
dynamic_note_score8 = calculate_similarity_dynamic(a_input, a_target)

print(f"input: {input_hangyul1} | target: {target_hangyul2} | 곡의 셈여림 유사도 | {dynamic_note_score1}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 곡의 셈여림 유사도 | {dynamic_note_score2}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 곡의 셈여림 유사도 | {dynamic_note_score3}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 곡의 셈여림 유사도 | {dynamic_note_score4}")
print(f"input: {input_file} | target: {target_file} | 곡의 셈여림 유사도 | {dynamic_note_score5}")
print(f"input: {input_file2} | target: {target_file} | 곡의 셈여림 유사도 | {dynamic_note_score6}")
print(f"input: {input_file3} | target: {target_file} | 곡의 셈여림 유사도 | {dynamic_note_score7}")
print(f"input: {a_input} | target: {a_target} | 곡의 셈여림 유사도 | {dynamic_note_score8}")

input: input2_1.csv | target: target2.csv | 곡의 셈여림 유사도 | 84.42%
input: input2_2.csv | target: target2.csv | 곡의 셈여림 유사도 | 76.00%
input: input2_3.csv | target: target2.csv | 곡의 셈여림 유사도 | 59.26%
input: target2.csv | target: target2.csv | 곡의 셈여림 유사도 | 100.00%
input: hello1.csv | target: hello2.csv | 곡의 셈여림 유사도 | 81.59%
input: hello3.csv | target: hello2.csv | 곡의 셈여림 유사도 | 73.45%
input: hello4.csv | target: hello2.csv | 곡의 셈여림 유사도 | 3.97%
input: a_input.csv | target: a_target.csv | 곡의 셈여림 유사도 | 98.35%


## 3_3_2. 셈여림의 변화 일관성

In [1328]:
def calculate_dynamic_change_consistency(input_file, target_file):
    df_input = pd.read_csv(input_file)
    df_target = pd.read_csv(target_file)

    dynamic_series_input = df_input['dynamic']
    dynamic_series_target = df_target['dynamic']

    dynamic_states_input = zip(dynamic_series_input, dynamic_series_input.shift(-1))
    dynamic_states_target = zip(dynamic_series_target, dynamic_series_target.shift(-1))

    # 상태 바뀌는 부분 빈도수 계산
    dynamic_counter_input = pd.Series(dynamic_states_input).value_counts().to_dict()
    dynamic_counter_target = pd.Series(dynamic_states_target).value_counts().to_dict()

    # 입력 파일과 정답 파일에서 공통으로 나타나는 공통 패턴 찾기
    common_patterns = {state: min(dynamic_counter_input.get(state, 0), dynamic_counter_target.get(state, 0)) for state in set(dynamic_counter_input) | set(dynamic_counter_target)}

    # 일관성 백분율 계산
    total_common_patterns = sum(common_patterns.values())
    total_input_patterns = sum(dynamic_counter_input.values())
    dynamic_change_consistency_percentage = min(100, (total_common_patterns / total_input_patterns) * 100)

    return f'{dynamic_change_consistency_percentage:.2f}%'

In [1329]:
# 셈여림의 변화
dynamic_change_score1 = calculate_dynamic_change_consistency(input_hangyul1, target_hangyul2)
dynamic_change_score2 = calculate_dynamic_change_consistency(input_hangyul2, target_hangyul2)
dynamic_change_score3 = calculate_dynamic_change_consistency(input_hangyul3, target_hangyul2)
dynamic_change_score4 = calculate_dynamic_change_consistency(target_hangyul2, target_hangyul2)
dynamic_change_score5 = calculate_dynamic_change_consistency(input_file, target_file)
dynamic_change_score6 = calculate_dynamic_change_consistency(input_file2, target_file)
dynamic_change_score7 = calculate_dynamic_change_consistency(input_file3, target_file)
dynamic_change_score8 = calculate_dynamic_change_consistency(a_input, a_target)

print(f"input: {input_hangyul1} | target: {target_hangyul2} | 곡의 셈여림 변화 일관성 점수 | {dynamic_change_score1}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 곡의 셈여림 변화 일관성 점수 | {dynamic_change_score2}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 곡의 셈여림 변화 일관성 점수 | {dynamic_change_score3}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 곡의 셈여림 변화 일관성 점수 | {dynamic_change_score4}")
print(f"input: {input_file} | target: {target_file} | 곡의 셈여림 변화 일관성 점수 | {dynamic_change_score5}")
print(f"input: {input_file2} | target: {target_file} | 곡의 셈여림 변화 일관성 점수 | {dynamic_change_score6}")
print(f"input: {input_file3} | target: {target_file} | 곡의 셈여림 변화 일관성 점수 | {dynamic_change_score7}")
print(f"input: {a_input} | target: {a_target} | 곡의 셈여림 변화 일관성 점수 | {dynamic_change_score8}")

input: input2_1.csv | target: target2.csv | 곡의 셈여림 변화 일관성 점수 | 82.19%
input: input2_2.csv | target: target2.csv | 곡의 셈여림 변화 일관성 점수 | 67.12%
input: input2_3.csv | target: target2.csv | 곡의 셈여림 변화 일관성 점수 | 56.16%
input: target2.csv | target: target2.csv | 곡의 셈여림 변화 일관성 점수 | 100.00%
input: hello1.csv | target: hello2.csv | 곡의 셈여림 변화 일관성 점수 | 79.82%
input: hello3.csv | target: hello2.csv | 곡의 셈여림 변화 일관성 점수 | 70.23%
input: hello4.csv | target: hello2.csv | 곡의 셈여림 변화 일관성 점수 | 1.78%
input: a_input.csv | target: a_target.csv | 곡의 셈여림 변화 일관성 점수 | 98.34%


## 3_4_1. 곡의 빠르기 유사도 점수 구하기
- 평균 템포 차이가 작을수록(두 트랙의 평균 템포가 비슷할수록) 높음

In [1333]:
def calculate_tempo_similarity_score(input_file, target_file):
    input_df = pd.read_csv(input_file)
    target_df = pd.read_csv(target_file)

    # 전체 트랙의 평균 템포 계산
    input_mean_tempo = input_df['tempo'].mean()
    target_mean_tempo = target_df['tempo'].mean()

    # 두 트랙 간의 평균 템포 차이 계산
    tempo_difference = abs(input_mean_tempo - target_mean_tempo)

    # 가중값 계산 (예시로 1.5를 사용, 차이가 클수록 가중값을 더 크게 설정할 수 있음)
    weight = 1.5
    weighted_tempo_difference = tempo_difference * weight

    max_similarity_score = 100  # 최대 유사도가 100%가 되도록 정규화

    # 가중값을 적용하여 유사도 점수 계산
    tempo_similarity_score = max(0, max_similarity_score - weighted_tempo_difference)

    return f'{tempo_similarity_score:.2f}%'

# 템포 유사도 Score 계산
tempo_score1 = calculate_tempo_similarity_score(input_hangyul1, target_hangyul2)
tempo_score2 = calculate_tempo_similarity_score(input_hangyul2, target_hangyul2)
tempo_score3 = calculate_tempo_similarity_score(input_hangyul3, target_hangyul2)
tempo_score4 = calculate_tempo_similarity_score(target_hangyul2, target_hangyul2)
tempo_score5 = calculate_tempo_similarity_score(input_file, target_file)
tempo_score6 = calculate_tempo_similarity_score(input_file2, target_file)
tempo_score7 = calculate_tempo_similarity_score(input_file3, target_file)
tempo_score8 = calculate_tempo_similarity_score(a_input, a_target)

print(f"input: {input_hangyul1} | target: {target_hangyul2} | 곡의 빠르기 유사도 Score1 | {tempo_score1}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 곡의 빠르기 유사도 Score2 | {tempo_score2}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 곡의 빠르기 유사도 Score3 | {tempo_score3}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 곡의 빠르기 유사도 Score4 | {tempo_score4}")
print(f"input: {input_file} | target: {target_file} | 곡의 빠르기 유사도 Score5 | {tempo_score5}")
print(f"input: {input_file2} | target: {target_file} | 곡의 빠르기 유사도 Score6 | {tempo_score6}")
print(f"input: {input_file3} | target: {target_file} | 곡의 빠르기 유사도 Score7 | {tempo_score7}")
print(f"input: {a_input} | target: {a_target} | 곡의 빠르기 유사도 Score8 | {tempo_score8}")

input: input2_1.csv | target: target2.csv | 곡의 빠르기 유사도 Score1 | 95.07%
input: input2_2.csv | target: target2.csv | 곡의 빠르기 유사도 Score2 | 85.21%
input: input2_3.csv | target: target2.csv | 곡의 빠르기 유사도 Score3 | 79.04%
input: target2.csv | target: target2.csv | 곡의 빠르기 유사도 Score4 | 100.00%
input: hello1.csv | target: hello2.csv | 곡의 빠르기 유사도 Score5 | 59.25%
input: hello3.csv | target: hello2.csv | 곡의 빠르기 유사도 Score6 | 65.00%
input: hello4.csv | target: hello2.csv | 곡의 빠르기 유사도 Score7 | 0.00%
input: a_input.csv | target: a_target.csv | 곡의 빠르기 유사도 Score8 | 100.00%


In [1225]:
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 곡의 음정 정확도 Score1 | {note_score1}")
print('-'*100)
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 곡의 음정 정확도 Score2 | {note_score2}")
print('-'*100)
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 곡의 음정 정확도 Score3 | {note_score3}")
print('-'*100)
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 곡의 음정 정확도 Score4 | {note_score4}")
print('-'*100)
print(f"input: {input_file} | target: {target_file} | 곡의 음정 정확도 Score5 | {note_score5}")
print('-'*100)
print(f"input: {input_file2} | target: {target_file} | 곡의 음정 정확도 Score6 | {note_score6}")
print('-'*100)
print(f"input: {input_file3} | target: {target_file} | 곡의 음정 정확도 Score7 | {note_score7}")
print('-'*100)
print(f"input: {a_input} | target: {a_target} | 곡의 음정 정확도 Score8 | {note_score8}")

input: input2_1.csv | target: target2.csv | 곡의 음정 정확도 Score1 | 84.25%
----------------------------------------------------------------------------------------------------
input: input2_2.csv | target: target2.csv | 곡의 음정 정확도 Score2 | 36.30%
----------------------------------------------------------------------------------------------------
input: input2_3.csv | target: target2.csv | 곡의 음정 정확도 Score3 | 41.78%
----------------------------------------------------------------------------------------------------
input: target2.csv | target: target2.csv | 곡의 음정 정확도 Score4 | 100.00%
----------------------------------------------------------------------------------------------------
input: hello1.csv | target: hello2.csv | 곡의 음정 정확도 Score5 | 25.50%
----------------------------------------------------------------------------------------------------
input: hello3.csv | target: hello2.csv | 곡의 음정 정확도 Score6 | 20.42%
---------------------------------------------------------------------------------

## 3_4_2. 빠르기의 변화

## 3_5. 붙임줄, 스타카토, 테누토, 늘임표

## 3_6. 악센트

- https://github.com/dalabdgw/Research_and_Experimental_Results/blob/main/Piano%20Performance%20Evaluation/MinseoPark/comparison.ipynb
- 민서 구현

## 3_7. 옥타브

In [1226]:
# def calculate_octave_accuracy(input_file, target_file):
#     df_input = pd.read_csv(input_file)
#     df_target = pd.read_csv(target_file)

#     total_matched_octaves = 0
#     total_notes = 0

#     for index in range(len(df_input)):
#         notes_input = eval(df_input.at[index, 'note'])
#         notes_target = eval(df_target.at[index, 'note'])

#         # 0으로 된 값 리스트로 처리
#         if not isinstance(notes_input, list):
#             notes_input = [notes_input]
#         if not isinstance(notes_target, list):
#             notes_target = [notes_target]

#         # 출력: 비교되는 옥타브 정보
#         input_octaves = [note // 12 for note in notes_input]
#         target_octaves = [note // 12 for note in notes_target]
#         print('-' * 100)
#         print(f"입력 데이터인 {input_file}의 Octaves: {input_octaves}")
#         print(f"정답 데이터인 {target_file}의 Octaves: {target_octaves}")
            
#         # 현재 행의 일치하는 옥타브 수 초기화
#         matched_octaves = 0

#         # 한 행에 있는 리스트들 간의 비교
#         # 수정된 부분: 모든 입력 옥타브가 정답 데이터에 있을 때만 점수 추가
#         if set(input_octaves).issubset(set(target_octaves)) and set(target_octaves).issubset(set(input_octaves)):
#             matched_octaves += 1
#             print(f'{set(input_octaves)}과 {set(target_octaves)} 일치!')
#             print('점수 1점 추가!!!')

#         total_matched_octaves += matched_octaves
#         total_notes += len(set(input_octaves) | set(target_octaves))
#         # 중복을 제외한 전체 옥타브 수

#     # 전체 옥타브 일치도 계산
#     max_possible_matched_octaves = total_notes  # 모든 노트의 옥타브가 일치하는 경우
#     octave_accuracy = min(100, (total_matched_octaves / max_possible_matched_octaves) * 100)

#     return f'{octave_accuracy:.2f}%'

# # 옥타브 정확도 계산
# octave_accuracy1 = calculate_octave_accuracy(input_hangyul1, target_hangyul2)
# octave_accuracy2 = calculate_octave_accuracy(input_hangyul2, target_hangyul2)
# octave_accuracy3 = calculate_octave_accuracy(input_hangyul3, target_hangyul2)
# octave_accuracy4 = calculate_octave_accuracy(target_hangyul2, target_hangyul2)
# octave_accuracy5 = calculate_octave_accuracy(input_file, target_file)
# octave_accuracy6 = calculate_octave_accuracy(input_file2, target_file)
# octave_accuracy7 = calculate_octave_accuracy(input_file3, target_file)
# octave_accuracy8 = calculate_octave_accuracy(a_input, a_target)

# print(f"input: {input_hangyul1} | target: {target_hangyul2} | 두 파일 간의 옥타브 정확도: {octave_accuracy1}")
# print(f"input: {input_hangyul2} | target: {target_hangyul2} | 두 파일 간의 옥타브 정확도: {octave_accuracy2}")
# print(f"input: {input_hangyul3} | target: {target_hangyul2} | 두 파일 간의 옥타브 정확도: {octave_accuracy3}")
# print(f"input: {target_hangyul2} | target: {target_hangyul2} | 두 파일 간의 옥타브 정확도: {octave_accuracy4}")
# print(f"input: {input_file} | target: {target_file} | 두 파일 간의 옥타브 정확도: {octave_accuracy5}")
# print(f"input: {input_file2} | target: {target_file} | 두 파일 간의 옥타브 정확도: {octave_accuracy6}")
# print(f"input: {input_file3} | target: {target_file} | 두 파일 간의 옥타브 정확도: {octave_accuracy7}")
# print(f"input: {a_input} | target: {a_target} | 두 파일 간의 옥타브 정확도: {octave_accuracy8}")

----------------------------------------------------------------------------------------------------
입력 데이터인 input2_1.csv의 Octaves: [6, 6]
정답 데이터인 target2.csv의 Octaves: [6]
{6}과 {6} 일치!
점수 1점 추가!!!
----------------------------------------------------------------------------------------------------
입력 데이터인 input2_1.csv의 Octaves: [6]
정답 데이터인 target2.csv의 Octaves: [6]
{6}과 {6} 일치!
점수 1점 추가!!!
----------------------------------------------------------------------------------------------------
입력 데이터인 input2_1.csv의 Octaves: [6]
정답 데이터인 target2.csv의 Octaves: [6]
{6}과 {6} 일치!
점수 1점 추가!!!
----------------------------------------------------------------------------------------------------
입력 데이터인 input2_1.csv의 Octaves: [6]
정답 데이터인 target2.csv의 Octaves: [6]
{6}과 {6} 일치!
점수 1점 추가!!!
----------------------------------------------------------------------------------------------------
입력 데이터인 input2_1.csv의 Octaves: [6]
정답 데이터인 target2.csv의 Octaves: [6, 6]
{6}과 {6} 일치!
점수 1점 추가!!!
-------------------

In [1334]:
def calculate_euclidean_distance(notes1, notes2):
    max_length = max(len(notes1), len(notes2))
    notes1 += [0] * (max_length - len(notes1))
    notes2 += [0] * (max_length - len(notes2))

    # 두 리스트 간의 유클리드 거리 계산
    distance = np.linalg.norm(np.array(notes1) - np.array(notes2))

    return distance

def calculate_octave_similarity(input_file, target_file):
    df_input = pd.read_csv(input_file)
    df_target = pd.read_csv(target_file)

    total_matched_octaves = 0
    total_notes = 0

    for index in range(len(df_input)):
        notes_input = eval(df_input.at[index, 'note'])
        notes_target = eval(df_target.at[index, 'note'])

        # 0으로 된 값 리스트로 처리
        if not isinstance(notes_input, list):
            notes_input = [notes_input]
        if not isinstance(notes_target, list):
            notes_target = [notes_target]

        # 출력: 비교되는 옥타브 정보
        input_octaves = list(set(note // 12 for note in notes_input))
        target_octaves = list(set(note // 12 for note in notes_target))
        print('-' * 100)
        print(f"입력 데이터인 {input_file}의 Octaves: {input_octaves}")
        print(f"정답 데이터인 {target_file}의 Octaves: {target_octaves}")
            
        # 현재 행의 일치하는 옥타브 수 초기화
        matched_octaves = 0

        # 한 행에 있는 리스트들 간의 비교
        # 수정된 부분: 유클리드 거리를 사용하여 옥타브 유사도 계산
        distance = calculate_euclidean_distance(input_octaves, target_octaves)
        similarity = 1 / (1 + distance)

        # 유사도가 일정 값 이상인 경우에 가중 점수 추가
        if similarity >= 0.8:
            matched_octaves += 1.5
            print(f'옥타브 유사도: {similarity:.2f} | 일치!')
            print('점수 2점 추가!!!')
        elif similarity >= 0.6:
            matched_octaves += 1.2
            print(f'옥타브 유사도: {similarity:.2f} | 일치!')
            print('점수 3점 추가!!!')
        elif similarity >= .5:
            matched_octaves += 1
            print(f'옥타브 유사도: {similarity:.2f} | 일치!')
            print('점수 1.5점 추가!!!')

        total_matched_octaves += matched_octaves
        total_notes += len(set(input_octaves) | set(target_octaves))
        # 중복을 제외한 전체 옥타브 수

    # 전체 옥타브 유사도 계산
    max_possible_matched_octaves = total_notes  # 모든 노트의 옥타브가 유사한 경우
    octave_similarity = min(100, (total_matched_octaves / max_possible_matched_octaves) * 100)

    return f'{octave_similarity:.2f}%'

# 옥타브 정확도 계산
octave_similarity1 = calculate_octave_similarity(input_hangyul1, target_hangyul2)
octave_similarity2 = calculate_octave_similarity(input_hangyul2, target_hangyul2)
octave_similarity3 = calculate_octave_similarity(input_hangyul3, target_hangyul2)
octave_similarity4 = calculate_octave_similarity(target_hangyul2, target_hangyul2)
octave_similarity5 = calculate_octave_similarity(input_file, target_file)
octave_similarity6 = calculate_octave_similarity(input_file2, target_file)
octave_similarity7 = calculate_octave_similarity(input_file3, target_file)
#octave_similarity8 = calculate_octave_similarity(a_input, a_target)

print(f"input: {input_hangyul1} | target: {target_hangyul2} | 두 파일 간의 옥타브 유사도: {octave_similarity1}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 두 파일 간의 옥타브 유사도: {octave_similarity2}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 두 파일 간의 옥타브 유사도: {octave_similarity3}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 두 파일 간의 옥타브 유사도: {octave_similarity4}")
print(f"input: {input_file} | target: {target_file} | 두 파일 간의 옥타브 유사도: {octave_similarity5}")
print(f"input: {input_file2} | target: {target_file} | 두 파일 간의 옥타브 유사도: {octave_similarity6}")
print(f"input: {input_file3} | target: {target_file} | 두 파일 간의 옥타브 유사도: {octave_similarity7}")
#print(f"input: {a_input} | target: {a_target} | 두 파일 간의 옥타브 유사도: {octave_similarity8}")

----------------------------------------------------------------------------------------------------
입력 데이터인 input2_1.csv의 Octaves: [6]
정답 데이터인 target2.csv의 Octaves: [6]
옥타브 유사도: 1.00 | 일치!
점수 2점 추가!!!
----------------------------------------------------------------------------------------------------
입력 데이터인 input2_1.csv의 Octaves: [6]
정답 데이터인 target2.csv의 Octaves: [6]
옥타브 유사도: 1.00 | 일치!
점수 2점 추가!!!
----------------------------------------------------------------------------------------------------
입력 데이터인 input2_1.csv의 Octaves: [6]
정답 데이터인 target2.csv의 Octaves: [6]
옥타브 유사도: 1.00 | 일치!
점수 2점 추가!!!
----------------------------------------------------------------------------------------------------
입력 데이터인 input2_1.csv의 Octaves: [6]
정답 데이터인 target2.csv의 Octaves: [6]
옥타브 유사도: 1.00 | 일치!
점수 2점 추가!!!
----------------------------------------------------------------------------------------------------
입력 데이터인 input2_1.csv의 Octaves: [6]
정답 데이터인 target2.csv의 Octaves: [6]
옥타브 유사도: 1.00 | 일치!
점수

In [1335]:
# 최종 코드

def calculate_euclidean_distance(notes1, notes2):
    max_length = max(len(notes1), len(notes2))
    notes1 += [0] * (max_length - len(notes1))
    notes2 += [0] * (max_length - len(notes2))

    # 두 리스트 간의 유클리드 거리 계산
    distance = np.linalg.norm(np.array(notes1) - np.array(notes2))

    return distance

def calculate_octave_similarity(input_file, target_file):
    df_input = pd.read_csv(input_file)
    df_target = pd.read_csv(target_file)

    total_matched_octaves = 0
    total_notes = 0

    for index in range(len(df_input)):
        notes_input = eval(df_input.at[index, 'note'])
        notes_target = eval(df_target.at[index, 'note'])

        # 0으로 된 값 리스트로 처리
        if not isinstance(notes_input, list):
            notes_input = [notes_input]
        if not isinstance(notes_target, list):
            notes_target = [notes_target]

        input_octaves = list(set(note // 12 for note in notes_input))
        target_octaves = list(set(note // 12 for note in notes_target))
            
        # 현재 행의 일치하는 옥타브 수 초기화
        matched_octaves = 0

        # 한 행에 있는 리스트들 간의 비교
        # 수정된 부분: 유클리드 거리를 사용하여 옥타브 유사도 계산
        distance = calculate_euclidean_distance(input_octaves, target_octaves)
        similarity = 1 / (1 + distance)

        # 유사도가 일정 값 이상인 경우에 가중 점수 추가
        if similarity >= 0.8:
            matched_octaves += 1.5
        elif similarity >= 0.6:
            matched_octaves += 1.2
        elif similarity >= .5:
            matched_octaves += 1

        total_matched_octaves += matched_octaves
        total_notes += len(set(input_octaves) | set(target_octaves))

    max_possible_matched_octaves = total_notes  # 모든 노트의 옥타브가 유사한 경우
    octave_similarity = min(100, (total_matched_octaves / max_possible_matched_octaves) * 100)

    return f'{octave_similarity:.2f}%'

In [1336]:
print('-'*100)
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 두 파일 간의 음정 정확도 Score1 | {note_score1}")
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 두 파일 간의 빠르기 유사도 Score1 | {tempo_score1}")
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 두 파일 간의 옥타브 유사도 Score1 | {octave_similarity1}")
print('-'*100)
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 두 파일 간의 음정 정확도 Score2 | {note_score2}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 두 파일 간의 빠르기 유사도 Score2 | {tempo_score2}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 두 파일 간의 옥타브 유사도 Score2 | {octave_similarity2}")
print('-'*100)
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 두 파일 간의 음정 정확도 Score3 | {note_score3}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 두 파일 간의 빠르기 유사도 Score3 | {tempo_score3}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 두 파일 간의 옥타브 유사도 Score3 | {octave_similarity3}")
print('-'*100)
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 두 파일 간의 음정 정확도 Score4 | {note_score4}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 두 파일 간의 빠르기 유사도 Score4 | {tempo_score4}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 두 파일 간의 옥타브 유사도 Score4 | {octave_similarity4}")
print('-'*100)
print(f"input: {input_file} | target: {target_file} | 두 파일 간의 음정 정확도 Score5 | {note_score5}")
print(f"input: {input_file} | target: {target_file} | 두 파일 간의 빠르기 유사도 Score5 | {tempo_score5}")
print(f"input: {input_file} | target: {target_file} | 두 파일 간의 옥타브 유사도 Score5 | {octave_similarity5}")
print('-'*100)
print(f"input: {input_file2} | target: {target_file} | 두 파일 간의 음정 정확도 Score6 | {note_score6}")
print(f"input: {input_file2} | target: {target_file} | 두 파일 간의 빠르기 유사도 Score6 | {tempo_score6}")
print(f"input: {input_file2} | target: {target_file} | 두 파일 간의 옥타브 유사도 Score6 | {octave_similarity6}")
print('-'*100)
print(f"input: {input_file3} | target: {target_file} | 두 파일 간의 음정 정확도 Score7 | {note_score7}")
print(f"input: {input_file3} | target: {target_file} | 두 파일 간의 빠르기 유사도 Score7 | {tempo_score7}")
print(f"input: {input_file3} | target: {target_file} | 두 파일 간의 옥타브 유사도 Score7 | {octave_similarity7}")
print('-'*100)
#print(f"input: {a_input} | target: {a_target} | 두 파일 간의 음정 정확도 Score8 | {note_score8}")
#print(f"input: {a_input} | target: {a_target} | 두 파일 간의 빠르기 유사도 Score8 | {tempo_score8}")
#print(f"input: {a_input} | target: {a_target} | 두 파일 간의 옥타브 유사도 Score8 | {octave_similarity8}")
#print('-'*100)

----------------------------------------------------------------------------------------------------
input: input2_1.csv | target: target2.csv | 두 파일 간의 음정 정확도 Score1 | 84.25%
input: input2_1.csv | target: target2.csv | 두 파일 간의 빠르기 유사도 Score1 | 95.07%
input: input2_1.csv | target: target2.csv | 두 파일 간의 옥타브 유사도 Score1 | 93.43%
----------------------------------------------------------------------------------------------------
input: input2_2.csv | target: target2.csv | 두 파일 간의 음정 정확도 Score2 | 36.30%
input: input2_2.csv | target: target2.csv | 두 파일 간의 빠르기 유사도 Score2 | 85.21%
input: input2_2.csv | target: target2.csv | 두 파일 간의 옥타브 유사도 Score2 | 62.61%
----------------------------------------------------------------------------------------------------
input: input2_3.csv | target: target2.csv | 두 파일 간의 음정 정확도 Score3 | 41.78%
input: input2_3.csv | target: target2.csv | 두 파일 간의 빠르기 유사도 Score3 | 79.04%
input: input2_3.csv | target: target2.csv | 두 파일 간의 옥타브 유사도 Score3 | 57.50%
----------------

## 3_8. 꾸밈음, 반복 기호

## 3_9. 리듬

## 3_10. 페달링

In [1317]:
def calculate_pedal_consistency(input_file, target_file):
    df_input = pd.read_csv(input_file)
    df_target = pd.read_csv(target_file)

    pedal_series_input = df_input['pedal']
    pedal_series_target = df_target['pedal']
    
    pedal_states_input = zip(pedal_series_input, pedal_series_input.shift(-1))
    pedal_states_target = zip(pedal_series_target, pedal_series_target.shift(-1))
        
    pedal_counter_input = pd.Series(pedal_states_input).value_counts().to_dict()
    pedal_counter_target = pd.Series(pedal_states_target).value_counts().to_dict()

    # 입력 파일과 정답 파일에서 공통으로 나타나는 공통 패턴 찾기
    common_patterns = {}

    for state in pedal_counter_input.keys() & pedal_counter_target.keys():
        common_patterns[state] = min(pedal_counter_target[state], pedal_counter_input[state] * 0.8)
        print(state)
        print(common_patterns)
        print('-'*100)

    total_common_patterns = sum(common_patterns.values())
    total_input_patterns = sum(pedal_counter_input.values())
    
    print(total_common_patterns)
    print(total_input_patterns)
    print('-'*100)
    
    # 일관성 백분율 계산
    pedal_consistency_percentage = min(100, (total_common_patterns / total_input_patterns) * 100)

    return pedal_consistency_percentage

input_file_path = 'river_flows_in_you_test_v2.csv'
target_file_path = 'river_flows_in_you_test_v3.csv'
consistency_percentage = calculate_pedal_consistency(input_file_path, target_file_path)
print(f'페달 일관성 점수: {consistency_percentage:.2f}%')

('0', '0')
{('0', '0'): 44722}
----------------------------------------------------------------------------------------------------
('[18, 18]', '0')
{('0', '0'): 44722, ('[18, 18]', '0'): 0.8}
----------------------------------------------------------------------------------------------------
('[2, 2]', '0')
{('0', '0'): 44722, ('[18, 18]', '0'): 0.8, ('[2, 2]', '0'): 0.8}
----------------------------------------------------------------------------------------------------
('[18, 18]', '[18, 18]')
{('0', '0'): 44722, ('[18, 18]', '0'): 0.8, ('[2, 2]', '0'): 0.8, ('[18, 18]', '[18, 18]'): 6}
----------------------------------------------------------------------------------------------------
('[0, 0]', '[0, 0]')
{('0', '0'): 44722, ('[18, 18]', '0'): 0.8, ('[2, 2]', '0'): 0.8, ('[18, 18]', '[18, 18]'): 6, ('[0, 0]', '[0, 0]'): 22}
----------------------------------------------------------------------------------------------------
('78', '0')
{('0', '0'): 44722, ('[18, 18]', '0'): 0.8, ('

In [1318]:
def calculate_pedal_consistency(input_file, target_file):
    df_input = pd.read_csv(input_file)
    df_target = pd.read_csv(target_file)

    pedal_series_input = df_input['pedal']
    pedal_series_target = df_target['pedal']
    
    pedal_states_input = zip(pedal_series_input, pedal_series_input.shift(-1))
    pedal_states_target = zip(pedal_series_target, pedal_series_target.shift(-1))
        
    pedal_counter_input = pd.Series(pedal_states_input).value_counts().to_dict()
    pedal_counter_target = pd.Series(pedal_states_target).value_counts().to_dict()

    # 입력 파일과 정답 파일에서 공통으로 나타나는 공통 패턴 찾기
    common_patterns = {}

    for state in pedal_counter_input.keys() & pedal_counter_target.keys():
        common_patterns[state] = min(pedal_counter_target[state], pedal_counter_input[state] * 0.8)

    total_common_patterns = sum(common_patterns.values())
    total_input_patterns = sum(pedal_counter_input.values())
    
    # 일관성 백분율 계산
    pedal_consistency_percentage = min(100, (total_common_patterns / total_input_patterns) * 100)

    return pedal_consistency_percentage

In [1319]:
input_file_path1 = 'river_flows_in_you_test_v2.csv'
target_file_path1 = 'river_flows_in_you_test_v3.csv'
input_file_path2 = 'river_flows_in_you_test_v3.csv'
target_file_path2 = 'river_flows_in_you_test_v2.csv'

pedal_consistency_percentage1 = calculate_pedal_consistency(input_file_path1, target_file_path1)
pedal_consistency_percentage2 = calculate_pedal_consistency(input_file_path2, target_file_path2)
print(f'페달 일관성 점수: {pedal_consistency_percentage1:.2f}%')
print(f'페달 일관성 점수: {pedal_consistency_percentage2:.2f}%')

페달 일관성 점수: 53.40%
페달 일관성 점수: 80.00%


## 3_11. 노트의 길이 유사도 점수 구하기
- 유클리드 거리를 계산하는 함수 활용
- 두 벡터가 비슷할수록 거리가 작아지고, 서로 다를수록 거리가 커짐
- 거리가 멀수록 1에 가까워지고, 거리가 가까울수록 0에 가까워짐
- 참고: https://wikidocs.net/24654

In [1231]:
def calculate_note_length_similarity(input_file, target_file):
    df_input = pd.read_csv(input_file)
    df_target = pd.read_csv(target_file)

    input_note_lengths = df_input['note_lengths'].values
    target_note_lengths = df_target['note_lengths'].values
    
    # 정규화를 통해 값의 범위를 [0, 1]로 변환
    normalized_input_lengths = (input_note_lengths - input_note_lengths.min()) / (input_note_lengths.max() - input_note_lengths.min())
    normalized_target_lengths = (target_note_lengths - target_note_lengths.min()) / (target_note_lengths.max() - target_note_lengths.min())

    # 유클리드 거리 계산
    distance = np.linalg.norm(normalized_input_lengths - normalized_target_lengths)

    # 거리 기반의 유사도 점수 계산
    max_distance = np.sqrt(10)  # 최대 거리 (정규화된 벡터의 길이가 10일 때를 최대거리로 둠)
    # 해당 값은 최종 값들의 평균이랑 가장 비슷한 결과가 나오는 값으로 정함
    similarity_score = max(0, 1 - distance / max_distance) * 100

    return f'{similarity_score:.2f}%'

# 유사도 계산
length_score1 = calculate_note_length_similarity(input_hangyul1, target_hangyul2)
length_score2 = calculate_note_length_similarity(input_hangyul2, target_hangyul2)
length_score3 = calculate_note_length_similarity(input_hangyul3, target_hangyul2)
length_score4 = calculate_note_length_similarity(target_hangyul2, target_hangyul2)
length_score5 = calculate_note_length_similarity(input_file, target_file)
length_score6 = calculate_note_length_similarity(input_file2, target_file)
length_score7 = calculate_note_length_similarity(input_file3, target_file)
#length_score8 = calculate_note_length_similarity(a_input, a_target)

# 결과 출력
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 노트 길이 유사도 Score1 | {length_score1}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 노트 길이 유사도 Score2 | {length_score2}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 노트 길이 유사도 Score3 | {length_score3}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 노트 길이 유사도 Score4 | {length_score4}")
print(f"input: {input_file} | target: {target_file} | 노트 길이 유사도 Score5 | {length_score5}")
print(f"input: {input_file2} | target: {target_file} | 노트 길이 유사도 Score6 | {length_score6}")
print(f"input: {input_file3} | target: {target_file} | 노트 길이 유사도 Score7 | {length_score7}")
#print(f"input: {a_input} | target: {a_target} | 노트 길이 유사도 Score8 | {length_score8}")

input: input2_1.csv | target: target2.csv | 노트 길이 유사도 Score1 | 73.00%
input: input2_2.csv | target: target2.csv | 노트 길이 유사도 Score2 | 70.83%
input: input2_3.csv | target: target2.csv | 노트 길이 유사도 Score3 | 70.82%
input: target2.csv | target: target2.csv | 노트 길이 유사도 Score4 | 100.00%
input: hello1.csv | target: hello2.csv | 노트 길이 유사도 Score5 | 21.63%
input: hello3.csv | target: hello2.csv | 노트 길이 유사도 Score6 | 26.70%
input: hello4.csv | target: hello2.csv | 노트 길이 유사도 Score7 | 5.62%


In [1330]:
print('-'*100)
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 두 파일 간의 음정 정확도 Score1 | {note_score1}")
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 두 파일 간의 빠르기 유사도 Score1 | {tempo_score1}")
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 두 파일 간의 옥타브 유사도 Score1 | {octave_similarity1}")
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 두 파일 간의 노트 길이 유사도 Score1 | {length_score1}")
print('-'*100)
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 두 파일 간의 음정 정확도 Score2 | {note_score2}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 두 파일 간의 빠르기 유사도 Score2 | {tempo_score2}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 두 파일 간의 옥타브 유사도 Score2 | {octave_similarity2}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 두 파일 간의 노트 길이 유사도 Score2 | {length_score2}")
print('-'*100)
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 두 파일 간의 음정 정확도 Score3 | {note_score3}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 두 파일 간의 빠르기 유사도 Score3 | {tempo_score3}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 두 파일 간의 옥타브 유사도 Score3 | {octave_similarity3}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 두 파일 간의 노트 길이 유사도 Score3 | {length_score3}")
print('-'*100)
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 두 파일 간의 음정 정확도 Score4 | {note_score4}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 두 파일 간의 빠르기 유사도 Score4 | {tempo_score4}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 두 파일 간의 옥타브 유사도 Score4 | {octave_similarity4}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 두 파일 간의 노트 길이 유사도 Score4 | {length_score4}")
print('-'*100)
print(f"input: {input_file} | target: {target_file} | 두 파일 간의 음정 정확도 Score5 | {note_score5}")
print(f"input: {input_file} | target: {target_file} | 두 파일 간의 빠르기 유사도 Score5 | {tempo_score5}")
print(f"input: {input_file} | target: {target_file} | 두 파일 간의 옥타브 유사도 Score5 | {octave_similarity5}")
print(f"input: {input_file} | target: {target_file} | 두 파일 간의 노트 길이 유사도 Score5 | {length_score5}")
print('-'*100)
print(f"input: {input_file2} | target: {target_file} | 두 파일 간의 음정 정확도 Score6 | {note_score6}")
print(f"input: {input_file2} | target: {target_file} | 두 파일 간의 빠르기 유사도 Score6 | {tempo_score6}")
print(f"input: {input_file2} | target: {target_file} | 두 파일 간의 옥타브 유사도 Score6 | {octave_similarity6}")
print(f"input: {input_file2} | target: {target_file} | 두 파일 간의 노트 길이 유사도 Score6 | {length_score6}")
print('-'*100)
print(f"input: {input_file3} | target: {target_file} | 두 파일 간의 음정 정확도 Score7 | {note_score7}")
print(f"input: {input_file3} | target: {target_file} | 두 파일 간의 빠르기 유사도 Score7 | {tempo_score7}")
print(f"input: {input_file3} | target: {target_file} | 두 파일 간의 옥타브 유사도 Score7 | {octave_similarity7}")
print(f"input: {input_file3} | target: {target_file} | 두 파일 간의 노트 길이 유사도 Score7 | {length_score7}")
print('-'*100)

----------------------------------------------------------------------------------------------------
input: input2_1.csv | target: target2.csv | 두 파일 간의 음정 정확도 Score1 | 84.25%
input: input2_1.csv | target: target2.csv | 두 파일 간의 빠르기 유사도 Score1 | 95.07%
input: input2_1.csv | target: target2.csv | 두 파일 간의 옥타브 유사도 Score1 | 93.43%
input: input2_1.csv | target: target2.csv | 두 파일 간의 노트 길이 유사도 Score1 | 73.00%
----------------------------------------------------------------------------------------------------
input: input2_2.csv | target: target2.csv | 두 파일 간의 음정 정확도 Score2 | 36.30%
input: input2_2.csv | target: target2.csv | 두 파일 간의 빠르기 유사도 Score2 | 85.21%
input: input2_2.csv | target: target2.csv | 두 파일 간의 옥타브 유사도 Score2 | 62.61%
input: input2_2.csv | target: target2.csv | 두 파일 간의 노트 길이 유사도 Score2 | 70.83%
----------------------------------------------------------------------------------------------------
input: input2_3.csv | target: target2.csv | 두 파일 간의 음정 정확도 Score3 | 41.78%
input: input

# 통합 테스트

In [1349]:
print('-'*100)
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 1. 두 파일 간의 음정 정확도 Score1 | {note_score1}")
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 2. 두 파일 간의 셈여림 유사도 Score1 | {dynamic_note_score1}")
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 3. 두 파일 간의 셈여림 변화 일관성 Score1 | {dynamic_change_score1}")
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 4. 두 파일 간의 빠르기 유사도 Score1 | {tempo_score1}")
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 8. 두 파일 간의 옥타브 유사도 Score1 | {octave_similarity1}")
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 11. 두 파일 간의 노트 길이 유사도 Score1 | {length_score1}")
print('-'*100)
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 1. 두 파일 간의 음정 정확도 Score2 | {note_score2}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 2. 두 파일 간의 셈여림 유사도 Score2 | {dynamic_note_score2}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 3. 두 파일 간의 셈여림 변화 일관성 Score2 | {dynamic_change_score2}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 4. 두 파일 간의 빠르기 유사도 Score2 | {tempo_score2}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 8. 두 파일 간의 옥타브 유사도 Score2 | {octave_similarity2}")
print(f"input: {input_hangyul2} | target: {target_hangyul2} | 11. 두 파일 간의 노트 길이 유사도 Score2 | {length_score2}")
print('-'*100)
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 1. 두 파일 간의 음정 정확도 Score3 | {note_score3}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 2. 두 파일 간의 셈여림 유사도 Score3 | {dynamic_note_score3}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 3. 두 파일 간의 셈여림 변화 일관성 Score3 | {dynamic_change_score3}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 4. 두 파일 간의 빠르기 유사도 Score3 | {tempo_score3}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 8. 두 파일 간의 옥타브 유사도 Score3 | {octave_similarity3}")
print(f"input: {input_hangyul3} | target: {target_hangyul2} | 11. 두 파일 간의 노트 길이 유사도 Score3 | {length_score3}")
print('-'*100)
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 1. 두 파일 간의 음정 정확도 Score4 | {note_score4}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 2. 두 파일 간의 셈여림 유사도 Score4 | {dynamic_note_score4}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 3. 두 파일 간의 셈여림 변화 일관성 Score4 | {dynamic_change_score4}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 4. 두 파일 간의 빠르기 유사도 Score4 | {tempo_score4}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 8. 두 파일 간의 옥타브 유사도 Score4 | {octave_similarity4}")
print(f"input: {target_hangyul2} | target: {target_hangyul2} | 11. 두 파일 간의 노트 길이 유사도 Score4 | {length_score4}")
print('-'*100)
print(f"input: {input_file} | target: {target_file} | 1. 두 파일 간의 음정 정확도 Score5 | {note_score5}")
print(f"input: {input_file} | target: {target_file} | 2. 두 파일 간의 셈여림 유사도 Score5 | {dynamic_note_score5}")
print(f"input: {input_file} | target: {target_file} | 3. 두 파일 간의 셈여림 변화 일관성 Score5 | {dynamic_change_score5}")
print(f"input: {input_file} | target: {target_file} | 4. 두 파일 간의 빠르기 유사도 Score5 | {tempo_score5}")
print(f"input: {input_file} | target: {target_file} | 8. 두 파일 간의 옥타브 유사도 Score5 | {octave_similarity5}")
print(f"input: {input_file} | target: {target_file} | 11. 두 파일 간의 노트 길이 유사도 Score5 | {length_score5}")
print('-'*100)
print(f"input: {input_file2} | target: {target_file} | 1. 두 파일 간의 음정 정확도 Score6 | {note_score6}")
print(f"input: {input_file2} | target: {target_file} | 2. 두 파일 간의 셈여림 유사도 Score6 | {dynamic_note_score6}")
print(f"input: {input_file2} | target: {target_file} | 3. 두 파일 간의 셈여림 변화 일관성 Score6 | {dynamic_change_score6}")
print(f"input: {input_file2} | target: {target_file} | 4. 두 파일 간의 빠르기 유사도 Score6 | {tempo_score6}")
print(f"input: {input_file2} | target: {target_file} | 8. 두 파일 간의 옥타브 유사도 Score6 | {octave_similarity6}")
print(f"input: {input_file2} | target: {target_file} | 11. 두 파일 간의 노트 길이 유사도 Score6 | {length_score6}")
print('-'*100)
print(f"input: {input_file3} | target: {target_file} | 1. 두 파일 간의 음정 정확도 Score7 | {note_score7}")
print(f"input: {input_file3} | target: {target_file} | 2. 두 파일 간의 셈여림 유사도 Score7 | {dynamic_note_score7}")
print(f"input: {input_file3} | target: {target_file} | 3. 두 파일 간의 셈여림 변화 일관성 Score7 | {dynamic_change_score7}")
print(f"input: {input_file3} | target: {target_file} | 4. 두 파일 간의 빠르기 유사도 Score7 | {tempo_score7}")
print(f"input: {input_file3} | target: {target_file} | 8. 두 파일 간의 옥타브 유사도 Score7 | {octave_similarity7}")
print(f"input: {input_file3} | target: {target_file} | 11. 두 파일 간의 노트 길이 유사도 Score7 | {length_score7}")
print('-'*100)

----------------------------------------------------------------------------------------------------
input: input2_1.csv | target: target2.csv | 1. 두 파일 간의 음정 정확도 Score1 | 84.25%
input: input2_1.csv | target: target2.csv | 2. 두 파일 간의 셈여림 유사도 Score1 | 84.42%
input: input2_1.csv | target: target2.csv | 3. 두 파일 간의 셈여림 변화 일관성 Score1 | 82.19%
input: input2_1.csv | target: target2.csv | 4. 두 파일 간의 빠르기 유사도 Score1 | 95.07%
input: input2_1.csv | target: target2.csv | 8. 두 파일 간의 옥타브 유사도 Score1 | 93.43%
input: input2_1.csv | target: target2.csv | 11. 두 파일 간의 노트 길이 유사도 Score1 | 73.00%
----------------------------------------------------------------------------------------------------
input: input2_2.csv | target: target2.csv | 1. 두 파일 간의 음정 정확도 Score2 | 36.30%
input: input2_2.csv | target: target2.csv | 2. 두 파일 간의 셈여림 유사도 Score2 | 76.00%
input: input2_2.csv | target: target2.csv | 3. 두 파일 간의 셈여림 변화 일관성 Score2 | 67.12%
input: input2_2.csv | target: target2.csv | 4. 두 파일 간의 빠르기 유사도 Score2 | 85.21%
i

In [1339]:
print('-'*100)
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 두 파일 간의 음정 정확도 Score1 | {note_score1}")
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 두 파일 간의 셈여림 유사도 Score1 | {dynamic_note_score1}")
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 두 파일 간의 셈여림 변화 일관성  Score1| {dynamic_change_score1}")
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 두 파일 간의 빠르기 유사도 Score1 | {tempo_score1}")
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 두 파일 간의 옥타브 유사도 Score1 | {octave_similarity1}")
print(f"input: {input_hangyul1} | target: {target_hangyul2} | 두 파일 간의 노트 길이 유사도 Score1 | {length_score1}")

----------------------------------------------------------------------------------------------------
input: input2_1.csv | target: target2.csv | 두 파일 간의 음정 정확도 Score1 | 84.25%
input: input2_1.csv | target: target2.csv | 두 파일 간의 셈여림 유사도 Score1 | 84.42%
input: input2_1.csv | target: target2.csv | 두 파일 간의 셈여림 변화 일관성  Score1| 82.19%
input: input2_1.csv | target: target2.csv | 두 파일 간의 빠르기 유사도 Score1 | 95.07%
input: input2_1.csv | target: target2.csv | 두 파일 간의 옥타브 유사도 Score1 | 93.43%
input: input2_1.csv | target: target2.csv | 두 파일 간의 노트 길이 유사도 Score1 | 73.00%


## 추가 작업해야 될 부분

In [1235]:
csv_path = './river_flows_in_you_test_v2.csv'
csv2_path = './river_flows_in_you_test_v3.csv'

df = pd.read_csv(csv_path)
df2 = pd.read_csv(csv2_path)

In [1236]:
for i in range(len(df)-1):
    if df['note'][i] == df['note'][i+1] or df['msg_type'][i] == '[]':
        df = df.drop(i)

for i in range(len(df2)-1):
    if df2['note'][i] == df2['note'][i+1] or df2['msg_type'][i] == '[]':
        df2 = df2.drop(i)

In [1237]:
df.shape, df2.shape

((1246, 15), (841, 15))

In [1238]:
print(df, '\n', df2)

           sec   tick   bpm                  msg_type channel      note  \
257      0.535    257  60.0               ['note_on']     [0]      [80]   
404      0.841    404  60.0               ['note_on']     [0]      [61]   
467      0.972    467  60.0  ['note_off', 'note_off']  [0, 0]  [61, 61]   
476      0.991    476  60.0               ['note_on']     [0]      [81]   
485      1.010    485  60.0               ['note_on']     [0]      [66]   
...        ...    ...   ...                       ...     ...       ...   
83555  174.072  83555  60.0  ['note_off', 'note_off']  [0, 0]  [57, 57]   
83624  174.216  83624  60.0  ['note_off', 'note_off']  [0, 0]  [81, 81]   
83652  174.275  83652  60.0  ['note_off', 'note_off']  [0, 0]  [62, 62]   
83830  174.645  83830  60.0  ['note_off', 'note_off']  [0, 0]  [69, 69]   
83894  174.779  83894  60.0                        []      []        []   

      velocity dynamic  accent  count  main_vol  depth   pedal  pan    tempo  
257       [36]      

In [1279]:
df.to_csv('./a_input.csv')
df2.to_csv('./a_target.csv')

In [1280]:
def calculate_note_accuracy(input_file, target_file):
    df_input = pd.read_csv(input_file)
    df_target = pd.read_csv(target_file)

    total_accuracy = 0

    for index in range(len(df_input)):
        notes_input = eval(df_input.at[index, 'note'])
        notes_target = eval(df_target.at[index, 'note'])

        # 0으로 된 값 리스트로 처리
        if isinstance(notes_input, int):
            notes_input = [notes_input]
        if isinstance(notes_target, int):
            notes_target = [notes_target]
        
        # 현재 행의 정확도 초기화
        row_accuracy = 0

        # 한 행에 있는 리스트들 간의 비교
        for i in range(len(notes_target)):
            for j in range(len(notes_input)):
                if notes_input[j] == notes_target[i]:
                    row_accuracy += 1
                    
        total_accuracy += row_accuracy

    # 전체 정확도 계산
    max_possible_accuracy = len(df_input)
    accuracy_percentage = min(100, (total_accuracy / max_possible_accuracy) * 100)

    return f'{accuracy_percentage:.2f}%'


note_score9 = calculate_note_accuracy(a_input, a_target)
print(f"{note_score9}")

KeyError: 841